# Image Embedding POC

### 1. Set Up

#### Permissions and environment variables

---
To host on Amazon SageMaker, we need to set up and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook as the AWS account role with SageMaker access. 

---

In [ ]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
print('Step completed.')

### 2. Select a model



#### Chose a model for Inference

In [ ]:
# model_version="*" fetches the latest version of the model
model_id, model_version = "tensorflow-icembedding-bit-m-r101x1-ilsvrc2012-featurevector-1", "*"
print('Step completed.')

### 3. Retrieve JumpStart Artifacts & Deploy an Endpoint


In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id)
model_predictor = my_model.deploy()

print(model_predictor.endpoint_name)
print('step completed.')

### 4. Download example images for inference
---
We download example images from the JumpStart assets S3 bucket.

---

In [ ]:
region = boto3.Session().region_name
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "inference-notebook-assets"
s3 = boto3.client("s3")


def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)


cat_jpg, dog_jpg = "cat.jpg", "dog.jpg"
download_from_s3(key_filenames=[cat_jpg, dog_jpg])

print('step completed.')

### 5. Query endpoint and parse response

---
Input to the endpoint is a single image in binary format. Output of the endpoint is a `json` with the image embedding.

---

In [ ]:
def query(model_predictor, image_file_name):
    """Query the model predictor."""

    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()

    query_response = model_predictor.predict(
        input_img_rb,
        {
            "ContentType": "application/x-image",
            "Accept": "application/json",
        },
    )
    return query_response

print('step completed.')

In [ ]:
# Your Code for to generate the first 5 elements of the embedding for both images 


## 6. Clean up the endpoint

In [ ]:
# Delete the SageMaker endpoint
model_predictor.delete_model()
model_predictor.delete_endpoint()